In [1]:
from openai import OpenAI
openai_client = OpenAI()

In [3]:
def llm(user_prompt, instructions=None, model="gpt-4o-mini"):
    messages = []

    if instructions:
        messages.append({
            "role":"system",
            "content":instructions
        })

    messages.append({
        "role":"user",
        "content":user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

In [4]:
llm("When does the course start?")

"Could you please specify which course you're inquiring about? Details like the institution or subject would be helpful!"

In [9]:
import requests

docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents=[]

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
from minsearch import Index

In [10]:
index = Index(
    text_fields=["questions","text","section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [19]:
def search (question):
    return index.search(
        question,
        boost_dict={'question':3.0,'section':0.3},
        num_results=5,
        filter_dict = {'course':'data-engineering-zoomcamp'}
    )

In [20]:
question = "I just discovered the course. can i njoin now?"

In [22]:
search_results = search(question)

In [23]:
instructions = """
You are a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
""".strip()

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

In [24]:
import json

In [27]:
print(search_results)

[{'text': "It's up to you which platform and environment you use for the course.\nGithub codespaces or GCP VM are just possible options, but you can do the entire course from your laptop.", 'section': 'General course-related questions', 'question': 'Environment - Do we really have to use GitHub codespaces? I already have PostgreSQL & Docker installed.', 'course': 'data-engineering-zoomcamp'}, {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-

In [28]:
def build_prompt(question, search_results):
    search_json=json.dumps(search_results),
    return prompt_template.format(question=question, context=search_json)

In [31]:
build_prompt(question, search_results)

'<QUESTION>\nI just discovered the course. can i njoin now?\n</QUESTION>\n\n<CONTEXT>\n(\'[{"text": "It\\\'s up to you which platform and environment you use for the course.\\\\nGithub codespaces or GCP VM are just possible options, but you can do the entire course from your laptop.", "section": "General course-related questions", "question": "Environment - Do we really have to use GitHub codespaces? I already have PostgreSQL & Docker installed.", "course": "data-engineering-zoomcamp"}, {"text": "The purpose of this document is to capture frequently asked technical questions\\\\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \\\\u201cOffice Hours\\\'\\\' live.1\\\\nSubscribe to course public Google Calendar (it works from Desktop only).\\\\nRegister before the course starts using this link.\\\\nJoin the course Telegram channel with announcements.\\\\nDon\\\\u2019t forget to register in DataTalks.Club\\\'s Slack and join the ch

In [73]:
def rag(question):
    search_results = search(question)
    user_prompt = build_prompt(question, search_results)
    return llm(user_prompt, instructions=instructions)

In [74]:
rag(question)

'The provided context does not specify whether enrollment is still open for the program. You may want to check the official course page or contact the course administrators for the most accurate information regarding enrollment.'

In [42]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [48]:
docs = [["I just discovered the course, can I still join?"],["I just found out about the program. Can I still enrol?"],
["you can join the course at any point of the time"]]
vectors=[]
for d in docs:
    
    v = embedding_model.encode(d)
    vectors.append(v)



In [50]:
q1, q2, d = vectors

In [53]:
q1 = q1[0]
q2=q2[0]
d=d[0]

In [56]:
q2.dot(d)

np.float32(0.50491434)

In [57]:
!uv add tqdm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 150 packages in 648ms                                       
Audited 145 packages in 179ms                                        


In [58]:
from tqdm.auto import tqdm

In [60]:
import numpy as np

In [61]:
embeddings=[]
for d in tqdm(documents):
    text=d['question']+' '+d['text']
    v = embedding_model.encode(text)
    embeddings.append(v)
embeddings = np.array(embeddings)

  0%|          | 0/948 [00:00<?, ?it/s]

In [62]:
embeddings.shape

(948, 768)

In [64]:
from minsearch import VectorSearch

In [66]:
vindex = VectorSearch(keyword_fields=['course'])
vindex.fit(embeddings, documents)

In [67]:
vindex.search(q2, filter_dict={'course':'data-engineering-zoomcamp'},num_results=5)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The

In [68]:
def vector_search(question):
    q = embedding_model.encode(question)
    return vindex.search(q,filter_dict={'course':'data-engineering-zoomcamp'},num_results=5)

In [69]:
vector_search("I just found out about the program. Can I still enrol?")

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The

In [70]:
def rag(question):
    search_results = vector_search(question)
    user_prompt = build_prompt(question, search_results)
    return llm(user_prompt, instructions=instructions)

In [72]:
question = "I just found out about the program. Can I still enrol?"
rag(question)

'Yes, you can still enroll in the program, but be aware that there will be deadlines for turning in the final projects.'

In [75]:
def hybrid_search(question):
    r1 = search(question)
    r2 = vector_search(question)
    return r1 + r2

In [76]:
def hy_rag(question):
    search_results = hybrid_search(question)
    user_prompt = build_prompt(question, search_results)
    return llm(user_prompt, instructions=instructions)

In [77]:
hy_rag(question)

"Yes, you can still join the course after the start date. Even if you don't register, you're eligible to submit the homework. However, be aware that there will be deadlines for turning in the final projects, so it’s advisable not to leave everything for the last minute."